In [ ]:
pip install scrapy

     |████████████████████████████████| 245kB 8.4MB/s 
     |████████████████████████████████| 3.1MB 21.5MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 3.2MB 44.6MB/s 
     |████████████████████████████████| 245kB 47.2MB/s 
     |████████████████████████████████| 3.2MB 24.9MB/s 
     |████████████████████████████████| 81kB 8.9MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11516 sha256=be0cdbf7ef72e8a3ce2759d9ce1ad48c7b415213241647284ec3c62d450bfdf4
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7766 sha256=0ede985448118bcc4f35e0020516f3b2c4f8a95f4731ba3be3d12369293b08e6
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
Successfully built PyDispatch

In [ ]:
# On importe la fonction 'get' (téléchargement) de 'requests' 
# Et la classe 'Selector' (Parsing) de 'scrapy'
from requests import get
from scrapy import Selector
# Lien de la page à scraper
url = "https://pubmed.ncbi.nlm.nih.gov/32472577/"
response = get(url)
source = None # Le code source de la page 
if response.status_code == 200 :
    # Si la requete s'est bien passee
    source = response.text

In [ ]:
if source :
    # Si le code source existe
    selector = Selector(text=source)
    titles = selector.css("div.toc ul > li")
    for title in titles:
        level = title.css("span.tocnumber::text").extract_first()
        name = title.css("span.toctext::text").extract_first()
        print(level + " " + name)

In [ ]:
source

'<!DOCTYPE html>\n\n\n\n\n<html lang="en" >\n<head  itemscope itemtype="http://schema.org/WebPage"  prefix="og: http://ogp.me/ns#" >\n    <meta charset="UTF-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n    <!-- Mobile properties -->\n    <meta name="HandheldFriendly" content="True">\n    <meta name="MobileOptimized" content="320">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n\n  \n  <link rel="preconnect" href="https://cdn.ncbi.nlm.nih.gov">\n  <link rel="preconnect" href="https://www.ncbi.nlm.nih.gov">\n  <link rel="preconnect" href="https://www.google-analytics.com">\n\n  \n  \n    <link rel="stylesheet" href="https://cdn.ncbi.nlm.nih.gov/pubmed/2447f27a-324e-4870-8feb-f8e445c68660/CACHE/css/output.38b2c95a9482.css" type="text/css">\n  \n\n  <link rel="stylesheet" href="https://cdn.ncbi.nlm.nih.gov/pubmed/2447f27a-324e-4870-8feb-f8e445c68660/CACHE/css/output.28abdc7cf11f.css" type="text/css">\n\n\n    <title>Alternatives to amyloid

In [ ]:
# API Key (created on Scraper API)
API_KEY = '6e2f13665a14d50e37acdfc9636b4877'

In [ ]:
def get_url(url):
    """ 
    Creation of the URL that will allow the legal scraping of Google results (use of the API key). 
    This URL is equivalent to a Google search.

    Parameter :
        url : google URL created from the keyword
    
    Out :
        proxy_url : URLs built using the API
    """

    payload = {'api_key': API_KEY, 'url': url, 'autoparse': 'true', 'country_code': 'fr', 'pws': 0}
    proxy_url = 'http://api.scraperapi.com/?' + urlencode(payload)
    date_url = proxy_url + link_sub_month(proxy_url, 6)
    return proxy_url

In [ ]:
class GoogleSpider(scrapy.Spider):
    """ 
    This class lists functions for scraping Google results from a list of keywords
    """

    # GoogleSpider class name
    name = 'pubmed'
    # Name of the site to be scraped
    allowed_domains = ['www.pubmed.ncbi.nlm.nih.gov']

    def parse(self, response):
        di = json.loads(response.text)
        pos = response.meta['pos']
        dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        for result in di['organic_results']:
            title = result['title']
            snippet = result['snippet']
            link = result['link']
            item = {'title': title, 'snippet': snippet, 'link': link, 'position': pos, 'date': dt}
            pos += 1
            yield item
        next_page = di['pagination']['nextPageUrl']
        if next_page:
            yield print(scrapy.Request(get_url(next_page), callback=self.parse, meta={'pos': pos}).text)

In [ ]:
list_source = []
i = 0

for index, row in df_result.iterrows():
    link = row['URL']
    query = row['Query']
    # 1 minute break to avoid API overloading
    time.sleep(60)
    # URL scraping
    response = get(link)

    # Test if the request was successful
    if response.status_code == 200:
        # Addition of the scraped google results and the corresponding query
        list_source.append([response.text, query])

        i+=1
        # Saving the results every 20 queries
        if (i%20==0):
            with open('etape'+str(i)+'.json', 'w') as jsonfile:
                json.dump(list_source, jsonfile)
            files.download('etape'+str(i)+'.json')

In [ ]:
import scrapy
from urllib.parse import urlencode
from urllib.parse import urlparse
import json
from datetime import datetime
API_KEY = '6e2f13665a14d50e37acdfc9636b4877'

In [ ]:
def get_url(url):
   payload = {'api_key': API_KEY, 'url': url, 'autoparse': 'true', 'country_code': 'fr'}
   proxy_url = 'http://api.scraperapi.com/?' + urlencode(payload)
   return proxy_url

def create_google_url(query, site=''):
   google_dict = {'q': query, 'num': 100, }
   if site:
       web = urlparse(site).netloc
       google_dict['as_sitesearch'] = web
       return 'https://pubmed.ncbi.nlm.nih.gov/?term=' + urlencode(google_dict)
   return 'https://pubmed.ncbi.nlm.nih.gov/?term=' + urlencode(google_dict)

queries = ['artificial intelligence', 'alzheimer']
class GoogleSpider(scrapy.Spider):
   name = 'pubmed'
   allowed_domains = ['www.pubmed.ncbi.nlm.nih.gov']
   custom_settings = {'ROBOTSTXT_OBEY': False, 'LOG_LEVEL': 'INFO', 'CONCURRENT_REQUESTS_PER_DOMAIN': 5, 'RETRY_TIMES': 0}
    
   def start_requests(self, queries):
     for query in queries:
      url = create_google_url(query)
      yield scrapy.Request(get_url(url), callback=self.parse, meta={'pos': 0}) 

   def parse(self, response):
       di = json.loads(response.text)
       pos = response.meta['pos']
       dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
       for result in di['organic_results']:
           title = result['title']
           snippet = result['snippet']
           link = result['link']
           item = {'title': title, 'snippet': snippet, 'link': link, 'position': pos, 'date': dt}
           pos += 1
           yield item
       next_page = di['pagination']['nextPageUrl']
       if next_page:
           yield print(scrapy.Request(get_url(next_page), callback=self.parse, meta={'pos': pos}).text)

In [ ]:
if __name__ == '__main__':
    result = list(GoogleSpider().start_requests(queries))

In [ ]:
result

[<GET http://api.scraperapi.com/?api_key=6e2f13665a14d50e37acdfc9636b4877&url=https%3A%2F%2Fpubmed.ncbi.nlm.nih.gov%2F%3Fterm%3Dq%3Dartificial%2Bintelligence%26num%3D100&autoparse=true&country_code=fr>,
 <GET http://api.scraperapi.com/?api_key=6e2f13665a14d50e37acdfc9636b4877&url=https%3A%2F%2Fpubmed.ncbi.nlm.nih.gov%2F%3Fterm%3Dq%3Dalzheimer%26num%3D100&autoparse=true&country_code=fr>]

In [ ]:
link = []
for i in range(len(result)):
    link = link + [str(result[i])[5:-1]]
print(link)

['http://api.scraperapi.com/?api_key=6e2f13665a14d50e37acdfc9636b4877&url=https%3A%2F%2Fpubmed.ncbi.nlm.nih.gov%2F%3Fterm%3Dq%3Dartificial%2Bintelligence%26num%3D100&autoparse=true&country_code=fr', 'http://api.scraperapi.com/?api_key=6e2f13665a14d50e37acdfc9636b4877&url=https%3A%2F%2Fpubmed.ncbi.nlm.nih.gov%2F%3Fterm%3Dq%3Dalzheimer%26num%3D100&autoparse=true&country_code=fr']


In [ ]:
# On importe la fonction 'get' (téléchargement) de 'requests' 
# Et la classe 'Selector' (Parsing) de 'scrapy'
from requests import get
from scrapy import Selector
# Lien de la page à scraper
url = link
response = []
for i in range(len(url)):
  response = response + [get(url[i])]
  source = [] # Le code source de la page 
  while response[i].status_code == 200 and i >= 0 :
    # Si la requete s'est bien passee
    source = source + [response[i].text]
    i = i - 1

In [ ]:
source[0]

'<!DOCTYPE html>\n\n\n\n\n<html lang="en" >\n<head  itemscope itemtype="http://schema.org/WebPage"  prefix="og: http://ogp.me/ns#" >\n    <meta charset="UTF-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n    <!-- Mobile properties -->\n    <meta name="HandheldFriendly" content="True">\n    <meta name="MobileOptimized" content="320">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n\n  \n  <link rel="preconnect" href="https://cdn.ncbi.nlm.nih.gov">\n  <link rel="preconnect" href="https://www.ncbi.nlm.nih.gov">\n  <link rel="preconnect" href="https://www.google-analytics.com">\n\n  \n  \n    <link rel="stylesheet" href="https://cdn.ncbi.nlm.nih.gov/pubmed/2447f27a-324e-4870-8feb-f8e445c68660/CACHE/css/output.38b2c95a9482.css" type="text/css">\n  \n\n  <link rel="stylesheet" href="https://cdn.ncbi.nlm.nih.gov/pubmed/2447f27a-324e-4870-8feb-f8e445c68660/CACHE/css/output.28abdc7cf11f.css" type="text/css">\n\n\n    <title>q=alzheimer - Search Re